In [ ]:
def scrollSlowly(driver):
    driver.execute_script("window.scrollTo(0, 500);")
    time.sleep(1)
    driver.execute_script("window.scrollTo(500, 800);")
    time.sleep(1)
    driver.execute_script("window.scrollTo(800, 1000);")
    time.sleep(2)
    driver.execute_script("window.scrollTo(1000, 1300);")
    time.sleep(2)
    driver.execute_script("window.scrollTo(1300, 1600);")
    time.sleep(1)
    

In [ ]:
import nltk 
temp = dict()
temp['Name'] = []
temp['Review text'] = []

driver.get('https://www.daraz.pk/products/redmi-12c-671-inch-display-4gb-ram-i425222112-s2014006595.html?spm=a2a0e.searchlist.sku.2.547b7a5aLl1JoT&search=1')
scrollSlowly(driver)
for page in range(0,10):
    try:
        reviews_container = driver.find_element(By.XPATH, '//*[@id="module_product_review"]/div/div/div[3]/div[1]')
    except NoSuchElementException:
        print('No Reviews')
        break
    reviews = reviews_container.find_elements(By.CLASS_NAME,'item')
    
    for i in range(len(reviews)):
        if reviews[i].find_element(By.CSS_SELECTOR, f"#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div:nth-child({i+1}) > div.item-content > div.content").text != '':
            temp['Name'].append(reviews[i].find_element(By.CLASS_NAME,'middle').find_element(By.TAG_NAME,'span').text[3:])
            temp['Review text'].append(reviews[i].find_element(By.CSS_SELECTOR, f"#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div:nth-child({i+1}) > div.item-content > div.content").text)
        
    try:
        if(page != 2):
            nxt_button = driver.find_element(By.XPATH, '//*[@id="module_product_review"]/div/div/div[3]/div[2]/div/button[2]')
            driver.execute_script("arguments[0].click();", nxt_button)
            time.sleep(5)
    except NoSuchElementException:
        print('No More Pages')
        break

In [ ]:
from nltk.corpus import stopwords
#storing all reviews list:
with open(r"C:\Users\mhass\Desktop\3rd Semester\Programming with AI\A6\pos.txt", 'r', encoding='utf-8') as posfile:
    content = posfile.read()
    pos_toks = nltk.sent_tokenize(content)

with open(r"C:\Users\mhass\Desktop\3rd Semester\Programming with AI\A6\neg.txt", 'r', encoding='utf-8') as negfile:
    content = negfile.read()
    neg_toks = nltk.sent_tokenize(content)

english_neg_pos = pd.read_excel(r"C:\Users\mhass\Desktop\3rd Semester\Programming with AI\A6\Positive and Negative Word List.xlsx")
cleaned_english=english_neg_pos.drop(columns='Unnamed: 0').dropna()    
negative_english = cleaned_english['Negative Sense Word List']
positive_english = cleaned_english['Positive Sense Word List']
def getNegWords():
    cleaned = neg_toks[0].split("\n")
    allPosWords = []
    for idx, word in enumerate(cleaned):
        word = word.split(':')
        allPosWords.append(word[1])
    return allPosWords

def getPostWords():
    cleaned = pos_toks[0].split("\n")
    allPosWords = []
    for idx, word in enumerate(cleaned):
        word = word.split(':')
        allPosWords.append(word[1])
    return allPosWords

In [ ]:
allPosWords = getPostWords()
for word in positive_english:
    allPosWords.append(word)
    
allNegWords = getNegWords()
for word in negative_english:
    allNegWords.append(word)

In [ ]:
#cleaning the reviews
import string
def cleanReviews(allreviews):
    reviews = reviewDF
    cleaned_revs = []
    reviews = []
    for review in allreviews:
        for rev_word in nltk.word_tokenize(review):
            if rev_word.lower() not in stopwords.words('english') and rev_word.lower() not in string.punctuation:
                cleaned_revs.append(rev_word)
        reviews.append(cleaned_revs)
    return reviews
                

In [ ]:
cleaned_revs = cleanReviews(reviewsDF['Review text'])
cleaned_revs

In [ ]:
def GetSentiment(revs):
    pos = 0
    neg = 0
    list_sent = []
    for review in revs:
        for review_word in review:
            if review_word in allPosWords:
                pos += 1
            elif review_word in allNegWords:
                neg += 1
        if pos > neg:
            list_sent.append('Positive')
        else:
            list_sent.append('Negative')
    return list_sent
        

In [ ]:
reviewsDF['Analysis']=GetSentiment(cleaned_revs)

In [ ]:
reviewsDF